In [ ]:
# Convolutional Auto encoder
# Importing reuiqred libraries
import os
import cv2
import numpy as np
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.metrics import MeanSquaredError
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Defining directories for images and sketches
img_dir = 'cuhk_training'
sketch_dir = 'cuhk_train_sketch'


img_rows, img_cols = 256, 256


In [ ]:
# Load dataset
X_train = []
Y_train = []

# Load and preprocess images and sketches
# Sorting the images
img_list = sorted(os.listdir(img_dir))
sketch_list = sorted(os.listdir(sketch_dir))
img_list.sort()
sketch_list.sort()
num_imgs = len(img_list)

# Looping over to get the images and sketches
for i in range(num_imgs):
  img_path = os.path.join(img_dir, img_list[i])
  sketch_path = os.path.join(sketch_dir, sketch_list[i])

  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  img = cv2.resize(img, (img_rows, img_cols))
  # Preprocessing: Apply Gaussian blur to input images to remove noise
  img = cv2.GaussianBlur(img, (5,5), 0)

  sketch = cv2.imread(sketch_path, cv2.IMREAD_GRAYSCALE)
  sketch = cv2.resize(sketch, (img_rows, img_cols))

  X_train.append(img)
  Y_train.append(sketch)

# Convert to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Reshape and normalize images
X_train = X_train.reshape(-1, img_rows, img_cols, 1) / 255.0
Y_train = Y_train.reshape(-1, img_rows, img_cols, 1) / 255.0



In [ ]:
# Define convolutional autoencoder architecture
input_img = Input(shape=(img_rows, img_cols, 1))

# Encoder
input_img = Input(shape=(img_rows, img_cols, 1))
# Encoder
x = Conv2D(256, (3, 3), activation='relu', padding='same')(input_img)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
# Hidden layer 2
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
# Hidden layer 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
# Hidden layer 4
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
# Hidden layer 6
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
# Hidden layer 7
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
# Autoencoder
autoencoder = Model(input_img, decoded)

# Learning rate
autoencoder.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=[MeanSquaredError()])



In [ ]:
# Train model
hist = autoencoder.fit(X_train, Y_train, epochs=100, batch_size=16, shuffle=True, validation_split=0.2)

# Plot the reconstruction loss over epochs
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])
plt.legend(["Training loss"],["Validaton loss"])
plt.title("Reconstruction Loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()


In [ ]:
# Generate sketches for new images
data_dir = '/content/drive/MyDrive/'
new_images_dir = os.path.join(data_dir, 'cuhk_test_photos')

output_dir = os.path.join(data_dir, 'cuhk_ae_out')
# output_dir = os.path.join(data_dir, 'cae_feret_trained_cuhk_out')

# Load test dataset
X_test = []

for img_file in os.listdir(new_images_dir):
    img_path = os.path.join(new_images_dir, img_file)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256, 256))  # resize image
    X_test.append(img)

# Convert to numpy arrays
X_test = np.array(X_test)

# Reshape and normalize images
X_test = X_test.reshape(-1, 256, 256, 1) / 255.0

# Generate and save sketches
for i in range(len(X_test)):
    sketch = autoencoder.predict(X_test[i].reshape(1, 256, 256, 1))
    sketch = sketch.reshape(256, 256) * 255.0
    sketch_path = os.path.join(output_dir, 'sketch_' + str(i) + '.jpg')
    cv2.imwrite(sketch_path, sketch)
    

In [ ]:
# Evaluation
Y_test = []
# Sorting the images
sketch_dirr = '/content/drive/MyDrive/cuhk_ae_test_out'
sketch_test = sorted(os.listdir(sketch_dirr))
sketch_test.sort()
num_imgs = len(img_list)

# Looping over to get the images and sketches
for i in range(100):
  sketch_path = os.path.join(sketch_dir, sketch_list[i])
  sketch = cv2.imread(sketch_path, cv2.IMREAD_GRAYSCALE)
  sketch = cv2.resize(sketch, (img_rows, img_cols))
  Y_test.append(sketch)

# Convert to numpy arrays
Y_test = np.array(Y_test)

# Reshape and normalize images
Y_test = Y_test.reshape(-1, img_rows, img_cols, 1) / 255.0

# Predict on the test set
Y_pred = autoencoder.predict(X_test)

# Calculating MSE
mse = np.mean((Y_test - Y_pred) ** 2)

# Plotting the MSE
plt.plot(hist.history['mean_squared_error'])
plt.plot(hist.history['val_mean_squared_error'])
plt.title('Mean Squared Error')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()
